# DSPy KNN few-shot example 
This noteboook shows how KNN few-shot can be implemented with DSPy using the **KNNFewShot** teleprompter. To illustrate, we use the HotPotQA dataset. Please see [intro.ipynb](../intro.ipynb) for other example use cases of DSPy.


In [1]:
import openai
import dspy
import json

In [2]:
with open("creds.json", "r") as creds:
    api_key = json.loads(creds.read())["openai_key"]

In [3]:
lm = dspy.OpenAI(model='gpt-4', api_key=api_key, model_type='chat', max_tokens = 500)
dspy.settings.configure(lm=lm)

In [4]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]


In [5]:
train_example = trainset[0]
print(train_example)
print(f"Question: {train_example.question}")
print(f"Answer: {train_example.answer}")

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt


In [6]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [7]:
class BasicQABot(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate = dspy.Predict(BasicQA)

    def forward(self,question):
        prediction = self.generate(question = question)
        return dspy.Prediction(answer = prediction.answer)

In [8]:
qa_bot = BasicQABot()
pred = qa_bot.forward("In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?")
pred.answer

'Alfred the Great'

In [9]:
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN

knn_teleprompter = KNNFewShot(KNN, 7, trainset)
compiled_knn = knn_teleprompter.compile(BasicQABot(), trainset=trainset)

In [10]:
example = devset[0]
pred = compiled_knn(question = example.question)
print("Question: ", example.question)
print("Expected answer: ", example.answer)
print("Prediction: ", pred.answer)

 57%|█████▋    | 4/7 [00:02<00:02,  1.44it/s]


Bootstrapped 4 full traces after 5 examples in round 0.
Question:  Are both Cangzhou and Qionghai in the Hebei province of China?
Expected answer:  no
Prediction:  No


In [11]:
lm.inspect_history(1)





Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: On the coast of what ocean is the birthplace of Diogal Sakho?
Answer: Atlantic Ocean

Question: Which is taller, the Empire State Building or the Bank of America Tower?
Answer: Empire State Building

Question: Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their field to go where?
Answer: Space

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Dar

Question: What is the code name for the German offensive that started this Second World War engagement on the Eastern Front (a few hundred kilometers from Moscow) between Soviet and German forces, which included 102nd Infantry Division?
Answer: Operation Citadel

Question: Which of these publications was most recently published, Who P

In [12]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_knn` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match

    
evaluate_on_hotpotqa(compiled_knn, metric)

 57%|█████▋    | 4/7 [00:03<00:02,  1.28it/s]


Bootstrapped 4 full traces after 5 examples in round 0.


 57%|█████▋    | 4/7 [00:03<00:02,  1.06it/s]  | 1/50 [00:03<03:01,  3.70s/it]


Bootstrapped 4 full traces after 5 examples in round 0.


 57%|█████▋    | 4/7 [00:02<00:01,  1.57it/s]  | 2/50 [00:08<03:14,  4.05s/it]


...

Bootstrapped 4 full traces after 5 examples in round 0.


Average Metric: 23 / 50  (46.0): 100%|██████████| 50/50 [03:26<00:00,  4.13s/it]
/home/jovyan/scdc/project-vaqa-autosuggest/Query-Generation-exploratory/dspy/dspy/evaluate/evaluate.py:126: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 23 / 50  (46.0%)


,question,example_answer,gold_titles,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}",Tyne River,❌ [False]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}",Alfred the Great,❌ [False]


(46.0,
           question  example_answer     gold_titles     pred_answer  answer_exact_match
 0   Are both Ca...              no  {Cangzhou, ...              No            True    
 1   Who conduct...  National Ho...  {2017–18 Pi...  National Ho...            True    
 2   The Wings e...   Steve Yzerman  {2006–07 De...   Steve Yzerman            True    
 3   What river ...  the River Tyne  {Crichton C...      Tyne River           False    
 4   In the 10th...  King Alfred...  {Æthelweard...  Alfred the ...           False    
 5   The Newark ...  Port Author...  {Newark Air...  Port Author...            True    
 6   Where did a...      Bundesliga  {Claudio Pi...            Peru           False    
 7   Are both Ch...              no  {Chico Muni...              No            True    
 8   In which Ma...  Waldo Count...  {Stockton S...    Waldo County           False    
 9   Which 90s r...  The Afghan ...  {Gene (band...  The Afghan ...            True    
 10  What year d...      